In [3]:
#This is the notebook for ML_Ex6_2

In [104]:
import pandas as pd
import numpy as np
import os
import re
from matplotlib import pyplot as plt
from scipy import optimize as opt
import scipy.io
import random
from sklearn import svm
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Yow-Gwo
[nltk_data]     Wang\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [136]:
with open(r"C:\Users\Yow-Gwo Wang\Desktop\ML\machine-learning-ex6\machine-learning-ex6\ex6\emailSample1.txt", 'r') as f:
    email = f.readlines()
email

['> Anyone knows how much it costs to host a web portal ?\n',
 '>\n',
 "Well, it depends on how many visitors you're expecting.\n",
 'This can be anywhere from less than 10 bucks a month to a couple of $100. \n',
 'You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 \n',
 'if youre running something big..\n',
 '\n',
 'To unsubscribe yourself from this mailing list, send an email to:\n',
 'groupname-unsubscribe@egroups.com\n',
 '\n']

In [134]:
with open(r"C:\Users\Yow-Gwo Wang\Desktop\ML\machine-learning-ex6\machine-learning-ex6\ex6\vocab.txt", 'r') as f:
    vocab=f.readlines()
    text = re.sub('[0-9]+','',''.join(vocab))
    text = re.sub('(http|https)://[^\s]*', 'httpaddr', text)
    text = re.sub('[^\s]+@[^\s]+', 'emailaddr', text)
    text = re.sub('[$]+', 'dollar', text)
    text = re.sub(r"[,.;@#?!&$]+\ *", " ", text)
    text = re.sub('[^a-zA-Z0-9]', ' ', text);
    text = re.sub(r"[,.;@#?!&$]+\ *", " ", text)
    text = re.sub(r"\s+", " ", text)
    vocab = text.split()

In [157]:
def processEmail(email_contents):
    word_indices = []
    text = [x.lower() for x in email_contents]
    text = re.sub('[0-9]+','number',''.join(text))
    text = re.sub('(http|https)://[^\s]*', 'httpaddr', text)
    text = re.sub('[^\s]+@[^\s]+', 'emailaddr', text)
    text = re.sub('[$]+', 'dollar', text)
    text = re.sub(r"[,.;@#?!&$]+\ *", " ", text)
    text = re.sub('[^a-zA-Z0-9]', ' ', text);
    text = re.sub(r"[,.;@#?!&$]+\ *", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = text.split(' ')
    ps = PorterStemmer()
    for idx, word in enumerate(text):
        text[idx] = ps1.stem(word)
    email_contents = text
    
    for word in email_contents:
        for idx, voc in enumerate(vocab):
            if word==voc:
                word_indices.append(idx+1)
    return word_indices

In [170]:
def emailFeatures(word_indices):
    x = np.zeros((1899, 1))
    for idx in word_indices:
        x[idx]=1
    
    return x

In [252]:
#Traing your spam prediction model
#train score and test score
train_email = scipy.io.loadmat(r"C:\Users\Yow-Gwo Wang\Desktop\ML\machine-learning-ex6\machine-learning-ex6\ex6\spamTrain.mat")
test_email = scipy.io.loadmat(r"C:\Users\Yow-Gwo Wang\Desktop\ML\machine-learning-ex6\machine-learning-ex6\ex6\spamTest.mat")
svc = svm.LinearSVC(C=0.1, loss='hinge', max_iter=1000)
svc.fit(train_email['X'], train_email['y'])
print(svc.score(train_email['X'], train_email['y']))
print(svc.score(test_email['Xtest'], test_email['ytest']))


0.9985
0.989


C:\Users\Yow-Gwo Wang\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [249]:
#Based on the model we traind, which word is likely classfied as spam
#Sort the index of the coeficient (which has higher possibility) from the vocab list
sorted_indices = np.argsort(svc.coef_, axis=None )[::-1]
for idx in sorted_indices[0:15]:
    print(vocab[idx])

our
click
remov
guarante
visit
basenumb
dollar
will
price
pleas
lo
nbsp
most
ga
hour


In [254]:
#email 1 classfied correctly
index = processEmail(email)
fe = emailFeatures(index)
conf = svc.predict(fe.transpose())
conf

array([0], dtype=uint8)

In [251]:
#email 2 classfied correctly
with open(r"C:\Users\Yow-Gwo Wang\Desktop\ML\machine-learning-ex6\machine-learning-ex6\ex6\emailSample2.txt", 'r') as f:
    email2 = f.readlines()
index2 = processEmail(email2)
fe2 = emailFeatures(index2)
conf = svc.predict(fe2.transpose())
conf

array([0], dtype=uint8)

In [234]:
#spam 1 mis-classfied
with open(r"C:\Users\Yow-Gwo Wang\Desktop\ML\machine-learning-ex6\machine-learning-ex6\ex6\spamSample1.txt", 'r') as f:
    spam1 = f.readlines()
index2 = processEmail(spam1)
fe2 = emailFeatures(index2)
conf = svc.predict(fe2.transpose())
conf

array([0], dtype=uint8)

In [250]:
#spam 2 classfied correctly
with open(r"C:\Users\Yow-Gwo Wang\Desktop\ML\machine-learning-ex6\machine-learning-ex6\ex6\spamSample2.txt", 'r') as f:
    spam2 = f.readlines()
index2 = processEmail(spam2)
fe2 = emailFeatures(index2)
conf = svc.predict(fe2.transpose())
conf

array([1], dtype=uint8)